# Space

In [1]:
import os
import sys 
import logging
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])

from recfldtkn.configfn import load_cohort_args
from config_observer.CF import cf_to_CaseFeatConfig
from config_observer.QCF import cf_to_QueryCaseFeatConfig
from config_observer.CKPD import ckpd_to_CkpdObsConfig
from recfldtkn.pipeline_dataset import pipeline_to_generate_dfcase_and_dataset


logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)
cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_args['ObsDTName'] = 'ObsDT'
cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace


[INFO:2024-04-21 09:17:27,114:(config.py@58 datasets)]: PyTorch version 2.1.2+cu121 available.


# CF Proc Settings

In [3]:
CASE_TAGGING_PROC_CONFIG = {
    'use_CF_from_disk': False,
    'use_CO_from_disk': False,
    'start_chunk_id': 0,
    'end_chunk_id': None,
    'chunk_size': 500000,
    'save_to_pickle': False,
    'num_processors': 1,
}


CASE_FIEDLING_PROC_CONFIG = {
    'use_CF_from_disk': False,
    'use_CO_from_disk': False,
    'start_chunk_id': 0,
    'end_chunk_id': None,
    'chunk_size': 100000,
    'num_processors': 1
}

In [4]:
SAVE_DF_CASE = False
SAVE_DS_DATA = False
LOAD_DF_CASE = False
LOAD_DS_DATA = False
SAVE_TRIGGER_DF = False # False
RANDOM_SAMPLE = 100

# Data Config

In [5]:
import pandas as pd 
CaseSetName = 'CGM5MinEntry'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case1 = pd.read_pickle(TriggerCasePath) # (1000)

CaseSetName = 'FoodEntryEOD'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case2 = pd.read_pickle(TriggerCasePath) # (1000)

df_case = pd.merge(df_case1, df_case2, how='inner')
# df_case = df_case1.sample(1000, random_state=0)
# df_case = df_case1.sample(1000, random_state=0).reset_index(drop = True)
df_case

,PID,ObsDT
0,1000037,2021-01-07
1,1000037,2021-01-08
2,1000048,2021-01-11
3,1000048,2021-01-13
4,1000048,2021-01-14
...,...,...
2701,3000039,2023-08-06
2702,3000039,2023-08-07
2703,3000039,2023-08-08
2704,3000039,2023-08-11


In [6]:
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'TestCaseSet.p')
print(TriggerCasePath)
df_case.to_pickle(TriggerCasePath)

../_Data/4-Data_CaseSet\TestCaseSet.p


In [7]:
ds_config_name = 'CgmGptMedalData'

In [8]:

if ds_config_name == 'CgmGptMedalData':
    # ################################################################
    # 0. ************ RFT config ************
    RecName_to_dsRec, RecName_to_dsRecInfo = {}, {}
    cohort_label_list = [1, 2, 3]
    # 1. ************ Case Trigger config ************
    TriggerCaseMethod = 'CGM5MinEntry'
    # 2. ************ InputCaseSetName ************
    # InputCaseSetName = 'sftcgmbf24haf2h-rs42-ds0.1-out0.1tstail0.1vd0.1'
    InputCaseSetName = 'TestCaseSet'
    # 3. ************ CaseTagging: TagMethod_List ************
    TagMethod_List = [
                      'PttBasicWD', 
                      'Bf24hCGMinfo', 'Af2hCGMinfo',
                    #   'Bf24hCGMrn', 'Af2hCGMrn',
                    #   'Bf24hCGMmode', 'Af2hCGMmode', 
                      
                      'Bf1mMEDALcf', 'Bf24hMEDALcf', 
                      'Bf2hMEDALcf', 'Af2hMEDALcf',
                      ]
    
    # 4. ************ CaseFiltering: FilterMethod_List ************
    FilterMethod_List = ['fPttBasicWD', 
                         'fBf24h280CGM',
                         'fAf2h24CGM', 
                         'fBf24HModePctn40', 
                         'fAf2HModePctn40',
                         ]
    # 5. ************ CaseSpliting: SplitDict ************
    SplitDict = {}
    # 6. ************ CaseSet Selection ************
    CaseSplitConfig = {}
    # 7. ************ CaseFeat: Feature Enriching ************
    CaseFeat_List =   [
                        'TargetCGM.Bf24H', 
                        'TargetCGM.Af2H',
                        
                        'TimeSparse.Bf24H', 
                        'TimeSparse.Af2H',
                        
                        'MedSparse.Bf24H', 
                        'MedSparse.Af2H',
                        
                        'EduSparse.Bf24H', 
                        'EduSparse.Af2H',
                        
                        'DietSparse.Bf24H', 
                        'DietSparse.Af2H',
                        
                        'ActivitySparse.Bf24H', 
                        'ActivitySparse.Af2H',
                        
                        'LabVitalSparse.Bf24H', 
                        'LabVitalSparse.Af2H',
                        ]

else:
    raise ValueError(f"Invalid ds_config_name: {ds_config_name}")


In [9]:
DataResults = pipeline_to_generate_dfcase_and_dataset(RecName_to_dsRec, 
                                                     RecName_to_dsRecInfo,

                                                    # df_case
                                                    InputCaseSetName,
                                                    TriggerCaseMethod,
                                                    TagMethod_List,
                                                    FilterMethod_List,
                                                    SplitDict,
                                                
                                                    # ds_case
                                                    CaseSplitConfig,
                                                    CaseFeat_List,

                                                    # config
                                                    cf_to_QueryCaseFeatConfig,
                                                    cf_to_CaseFeatConfig, 
                                                    SPACE,
                                                    cohort_args, 
                                                    cohort_label_list,
                                                    
                                                    # proc sets
                                                    CASE_TAGGING_PROC_CONFIG,
                                                    CASE_FIEDLING_PROC_CONFIG,
                                                    SAVE_DF_CASE,
                                                    SAVE_DS_DATA,
                                                    LOAD_DF_CASE, 
                                                    LOAD_DS_DATA,
                                                    RANDOM_SAMPLE,
                                                    SAVE_TRIGGER_DF,
                                                )

[INFO:2024-04-21 09:18:14,716:(pipeline_dataset.py@74 recfldtkn.pipeline_dataset)]: -------------- (0) RecName_to_dsRec: [] --------------
[INFO:2024-04-21 09:18:14,717:(pipeline_dataset.py@78 recfldtkn.pipeline_dataset)]: -------------- (1) TriggerCaseMethod: CGM5MinEntry --------------
[INFO:2024-04-21 09:18:14,726:(pipeline_dataset.py@84 recfldtkn.pipeline_dataset)]: -------------- (2) InputCaseSetName: TestCaseSet --------------
[INFO:2024-04-21 09:18:14,734:(pipeline_dataset.py@95 recfldtkn.pipeline_dataset)]: InputCaseSetName: TestCaseSet
[INFO:2024-04-21 09:18:14,734:(pipeline_dataset.py@96 recfldtkn.pipeline_dataset)]: df_case shape: (2706, 2)
[INFO:2024-04-21 09:18:14,734:(pipeline_dataset.py@130 recfldtkn.pipeline_dataset)]: ds_case path: ../_Data/4-Data_CaseSet\TestCaseSet-PttBasicWD.Bf24hCGMinfo.Af2hCGMinfo.Bf1mMEDALcf.Bf24hMEDALcf.Bf2hMEDALcf.Af2hMEDALcf-fPttBasicWD.fBf24h280CGM.fAf2h24CGM.fBf24HModePctn40.fAf2HModePctn40.p
[INFO:2024-04-21 09:18:14,735:(pipeline_dataset.p

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:18:28,408:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\3-RawData2023_CVSDeRxAug\ImpMed_data ...
[INFO:2024-04-21 09:18:28,657:(loadtools.py@156 recfldtkn.loadtools)]: No such folder: ../_Data/1-Data_RFT\1-RawData2022_CGM\LessonProg_data ...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

[INFO:2024-04-21 09:19:43,339:(pipeline_case.py@630 recfldtkn.pipeline_case)]: ds 75: finish one CaseFeat [ActivitySparse.Af2H] for splitname [inference]



[INFO:2024-04-21 09:19:43,340:(pipeline_case.py@631 recfldtkn.pipeline_case)]: 

++++++++ DONE ds: 75, splitname: inference, CaseFeat ActivitySparse.Af2H, 
[INFO:2024-04-21 09:19:43,340:(pipeline_case.py@632 recfldtkn.pipeline_case)]: the number of results in ds_case_split_cf_result_list: 1
[INFO:2024-04-21 09:19:43,340:(pipeline_case.py@639 recfldtkn.pipeline_case)]: Dataset({
    features: ['PID', 'ObsDT', 'ActivitySparse.Af2H.timestep', 'ActivitySparse.Af2H.input_ids', 'ActivitySparse.Af2H.input_wgts', 'ActivitySparse.Af2H.timeinfo', 'ActivitySparse.Af2H.timevalues'],
    num_rows: 75
})
[INFO:2024-04-21 09:19:43,343:(pipeline_case.py@604 recfldtkn.pipeline_case)]: 



----------------------ds 75: START one CaseFeat [LabVitalSparse.Bf24H] for splitname [inference]
[INFO:2024-04-21 09:19:43,343:(pipeline_case.py@605 recfldtkn.pip

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:19:45,561:(pipeline_case.py@630 recfldtkn.pipeline_case)]: ds 75: finish one CaseFeat [LabVitalSparse.Bf24H] for splitname [inference]



[INFO:2024-04-21 09:19:45,561:(pipeline_case.py@631 recfldtkn.pipeline_case)]: 

++++++++ DONE ds: 75, splitname: inference, CaseFeat LabVitalSparse.Bf24H, 
[INFO:2024-04-21 09:19:45,562:(pipeline_case.py@632 recfldtkn.pipeline_case)]: the number of results in ds_case_split_cf_result_list: 1
[INFO:2024-04-21 09:19:45,562:(pipeline_case.py@639 recfldtkn.pipeline_case)]: Dataset({
    features: ['PID', 'ObsDT', 'LabVitalSparse.Bf24H.timestep', 'LabVitalSparse.Bf24H.input_ids', 'LabVitalSparse.Bf24H.input_wgts', 'LabVitalSparse.Bf24H.timeinfo', 'LabVitalSparse.Bf24H.timevalues'],
    num_rows: 75
})
[INFO:2024-04-21 09:19:45,565:(pipeline_case.py@604 recfldtkn.pipeline_case)]: 



----------------------ds 75: START one CaseFeat [LabVitalSparse.Af2H] for splitname [inference]
[INFO:2024-04-21 09:19:45,565:(pipeline_case.py@605 recfldt

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

[INFO:2024-04-21 09:19:47,518:(pipeline_case.py@630 recfldtkn.pipeline_case)]: ds 75: finish one CaseFeat [LabVitalSparse.Af2H] for splitname [inference]



[INFO:2024-04-21 09:19:47,518:(pipeline_case.py@631 recfldtkn.pipeline_case)]: 

++++++++ DONE ds: 75, splitname: inference, CaseFeat LabVitalSparse.Af2H, 
[INFO:2024-04-21 09:19:47,519:(pipeline_case.py@632 recfldtkn.pipeline_case)]: the number of results in ds_case_split_cf_result_list: 1
[INFO:2024-04-21 09:19:47,519:(pipeline_case.py@639 recfldtkn.pipeline_case)]: Dataset({
    features: ['PID', 'ObsDT', 'LabVitalSparse.Af2H.timestep', 'LabVitalSparse.Af2H.input_ids', 'LabVitalSparse.Af2H.input_wgts', 'LabVitalSparse.Af2H.timeinfo', 'LabVitalSparse.Af2H.timevalues'],
    num_rows: 75
})
[INFO:2024-04-21 09:19:47,524:(pipeline_case.py@663 recfldtkn.pipeline_case)]: cf TargetCGM.Bf24H: ds_case_split_cf: ['TargetCGM.Bf24H.input_ids']
[INFO:2024-04-21 09:19:47,526:(pipeline_case.py@663 recfldtkn.pipeline_case)]: cf TargetCGM.Af2H: 

In [10]:
[i for i in DataResults]

['case_id_columns',
 'FinalOutCaseSetName',
 'df_case',
 'cf_to_CFVocab',
 'ds_case_cf_dict']

In [11]:
ds_case_cf_dict = DataResults['ds_case_cf_dict']
ds_case_cf_dict 

{'inference': Dataset({
     features: ['PID', 'ObsDT', 'TargetCGM.Bf24H.input_ids', 'TargetCGM.Af2H.input_ids', 'TimeSparse.Bf24H.timeinfo', 'TimeSparse.Bf24H.timevalues', 'TimeSparse.Af2H.timeinfo', 'TimeSparse.Af2H.timevalues', 'MedSparse.Bf24H.timestep', 'MedSparse.Bf24H.input_ids', 'MedSparse.Bf24H.input_wgts', 'MedSparse.Bf24H.timeinfo', 'MedSparse.Bf24H.timevalues', 'MedSparse.Af2H.timestep', 'MedSparse.Af2H.input_ids', 'MedSparse.Af2H.input_wgts', 'MedSparse.Af2H.timeinfo', 'MedSparse.Af2H.timevalues', 'EduSparse.Bf24H.timestep', 'EduSparse.Bf24H.input_ids', 'EduSparse.Bf24H.input_wgts', 'EduSparse.Bf24H.timeinfo', 'EduSparse.Bf24H.timevalues', 'EduSparse.Af2H.timestep', 'EduSparse.Af2H.input_ids', 'EduSparse.Af2H.input_wgts', 'EduSparse.Af2H.timeinfo', 'EduSparse.Af2H.timevalues', 'DietSparse.Bf24H.timestep', 'DietSparse.Bf24H.input_ids', 'DietSparse.Bf24H.input_wgts', 'DietSparse.Bf24H.timeinfo', 'DietSparse.Bf24H.timevalues', 'DietSparse.Af2H.timestep', 'DietSparse.Af2H.inpu

In [12]:
dataset = ds_case_cf_dict['inference']
dataset

Dataset({
    features: ['PID', 'ObsDT', 'TargetCGM.Bf24H.input_ids', 'TargetCGM.Af2H.input_ids', 'TimeSparse.Bf24H.timeinfo', 'TimeSparse.Bf24H.timevalues', 'TimeSparse.Af2H.timeinfo', 'TimeSparse.Af2H.timevalues', 'MedSparse.Bf24H.timestep', 'MedSparse.Bf24H.input_ids', 'MedSparse.Bf24H.input_wgts', 'MedSparse.Bf24H.timeinfo', 'MedSparse.Bf24H.timevalues', 'MedSparse.Af2H.timestep', 'MedSparse.Af2H.input_ids', 'MedSparse.Af2H.input_wgts', 'MedSparse.Af2H.timeinfo', 'MedSparse.Af2H.timevalues', 'EduSparse.Bf24H.timestep', 'EduSparse.Bf24H.input_ids', 'EduSparse.Bf24H.input_wgts', 'EduSparse.Bf24H.timeinfo', 'EduSparse.Bf24H.timevalues', 'EduSparse.Af2H.timestep', 'EduSparse.Af2H.input_ids', 'EduSparse.Af2H.input_wgts', 'EduSparse.Af2H.timeinfo', 'EduSparse.Af2H.timevalues', 'DietSparse.Bf24H.timestep', 'DietSparse.Bf24H.input_ids', 'DietSparse.Bf24H.input_wgts', 'DietSparse.Bf24H.timeinfo', 'DietSparse.Bf24H.timevalues', 'DietSparse.Af2H.timestep', 'DietSparse.Af2H.input_ids', 'DietSp

In [13]:
batch = dataset[:8]

In [14]:
[i for i in batch]

['PID',
 'ObsDT',
 'TargetCGM.Bf24H.input_ids',
 'TargetCGM.Af2H.input_ids',
 'TimeSparse.Bf24H.timeinfo',
 'TimeSparse.Bf24H.timevalues',
 'TimeSparse.Af2H.timeinfo',
 'TimeSparse.Af2H.timevalues',
 'MedSparse.Bf24H.timestep',
 'MedSparse.Bf24H.input_ids',
 'MedSparse.Bf24H.input_wgts',
 'MedSparse.Bf24H.timeinfo',
 'MedSparse.Bf24H.timevalues',
 'MedSparse.Af2H.timestep',
 'MedSparse.Af2H.input_ids',
 'MedSparse.Af2H.input_wgts',
 'MedSparse.Af2H.timeinfo',
 'MedSparse.Af2H.timevalues',
 'EduSparse.Bf24H.timestep',
 'EduSparse.Bf24H.input_ids',
 'EduSparse.Bf24H.input_wgts',
 'EduSparse.Bf24H.timeinfo',
 'EduSparse.Bf24H.timevalues',
 'EduSparse.Af2H.timestep',
 'EduSparse.Af2H.input_ids',
 'EduSparse.Af2H.input_wgts',
 'EduSparse.Af2H.timeinfo',
 'EduSparse.Af2H.timevalues',
 'DietSparse.Bf24H.timestep',
 'DietSparse.Bf24H.input_ids',
 'DietSparse.Bf24H.input_wgts',
 'DietSparse.Bf24H.timeinfo',
 'DietSparse.Bf24H.timevalues',
 'DietSparse.Af2H.timestep',
 'DietSparse.Af2H.input_ids

In [15]:
batch['DietSparse.Bf24H.input_ids']

[[[42,
   44,
   10,
   16,
   14,
   8,
   24,
   32,
   26,
   20,
   18,
   36,
   12,
   22,
   34,
   42,
   56,
   10,
   11,
   16,
   14,
   15,
   8,
   9,
   24,
   25,
   32,
   33,
   52,
   20,
   18,
   36,
   12,
   22,
   34,
   42,
   66,
   10,
   11,
   16,
   14,
   8,
   9,
   24,
   32,
   26,
   20,
   18,
   36,
   12,
   22,
   34,
   42,
   56,
   171,
   172,
   60,
   14,
   15,
   285,
   286,
   24,
   25,
   297,
   298,
   26,
   27,
   20,
   18,
   36,
   12,
   22,
   34,
   535,
   542,
   545,
   552,
   535,
   542,
   545,
   552,
   535,
   542,
   545,
   552,
   535,
   542,
   545,
   552],
  [42,
   66,
   10,
   11,
   16,
   78,
   79,
   285,
   24,
   25,
   297,
   298,
   52,
   53,
   20,
   18,
   36,
   173,
   22,
   34,
   35,
   42,
   66,
   126,
   16,
   14,
   15,
   8,
   9,
   24,
   25,
   297,
   298,
   26,
   27,
   50,
   18,
   19,
   36,
   12,
   22,
   23,
   34,
   35,
   42,
   30,
   31,
   215,
   216,
   16,
  

In [16]:
# batch['TimeSparse.Bf24H.timeinfo']

In [17]:
batch['TimeSparse.Bf24H.timevalues']

[['2021-05-31T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-06-12T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-08-23T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-09-03T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-09-24T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-09-30T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-10-22T00:00:00', '5Min', '5', 'minutes', '-288:To:0'],
 ['2021-11-05T00:00:00', '5Min', '5', 'minutes', '-288:To:0']]

# EntryFn

In [18]:
TASK = 'CgmMedalNtp'
ENTRY_METHOD = f'EntryFor{TASK}'


EntryArgs = {
    'Target_CFs': ['TargetCGM.Bf24H', 'TargetCGM.Af2H'],
    
    'MEDAL_Fields': {
        'Time': ['TimeSparse.Bf24H', 'TimeSparse.Af2H'],
        
        'Med':  ['MedSparse.Bf24H',  'MedSparse.Af2H'],
        # 'Med':  ['MedSparse.Bf24H',  'Mask.MedSparse.Af2H'],
        
        'Edu': ['EduSparse.Bf24H', 'EduSparse.Af2H'],
        # 'Edu': ['EduSparse.Bf24H', 'Mask.EduSparse.Af2H'],
        
        'Diet': ['DietSparse.Bf24H', 'DietSparse.Af2H'],
        # 'Diet': ['DietSparse.Bf24H', 'Mask.DietSparse.Af2H'],
        
        'Activity': ['ActivitySparse.Bf24H', 'ActivitySparse.Af2H'],
        # 'Activity': ['ActivitySparse.Bf24H', 'Mask.ActivitySparse.Af2H'],
        
        'Lab': ['LabVitalSparse.Bf24H', 'LabVitalSparse.Af2H'],
        # 'Lab': ['LabVitalSparse.Bf24H', 'Mask.LabVitalSparse.Af2H'],
    }, 
}


print(ENTRY_METHOD)

EntryForCgmMedalNtp


In [19]:
dataset

Dataset({
    features: ['PID', 'ObsDT', 'TargetCGM.Bf24H.input_ids', 'TargetCGM.Af2H.input_ids', 'TimeSparse.Bf24H.timeinfo', 'TimeSparse.Bf24H.timevalues', 'TimeSparse.Af2H.timeinfo', 'TimeSparse.Af2H.timevalues', 'MedSparse.Bf24H.timestep', 'MedSparse.Bf24H.input_ids', 'MedSparse.Bf24H.input_wgts', 'MedSparse.Bf24H.timeinfo', 'MedSparse.Bf24H.timevalues', 'MedSparse.Af2H.timestep', 'MedSparse.Af2H.input_ids', 'MedSparse.Af2H.input_wgts', 'MedSparse.Af2H.timeinfo', 'MedSparse.Af2H.timevalues', 'EduSparse.Bf24H.timestep', 'EduSparse.Bf24H.input_ids', 'EduSparse.Bf24H.input_wgts', 'EduSparse.Bf24H.timeinfo', 'EduSparse.Bf24H.timevalues', 'EduSparse.Af2H.timestep', 'EduSparse.Af2H.input_ids', 'EduSparse.Af2H.input_wgts', 'EduSparse.Af2H.timeinfo', 'EduSparse.Af2H.timevalues', 'DietSparse.Bf24H.timestep', 'DietSparse.Bf24H.input_ids', 'DietSparse.Bf24H.input_wgts', 'DietSparse.Bf24H.timeinfo', 'DietSparse.Bf24H.timevalues', 'DietSparse.Af2H.timestep', 'DietSparse.Af2H.input_ids', 'DietSp

In [20]:
CaseFeat_List

['TargetCGM.Bf24H',
 'TargetCGM.Af2H',
 'TimeSparse.Bf24H',
 'TimeSparse.Af2H',
 'MedSparse.Bf24H',
 'MedSparse.Af2H',
 'EduSparse.Bf24H',
 'EduSparse.Af2H',
 'DietSparse.Bf24H',
 'DietSparse.Af2H',
 'ActivitySparse.Bf24H',
 'ActivitySparse.Af2H',
 'LabVitalSparse.Bf24H',
 'LabVitalSparse.Af2H']

In [21]:
cf_to_CFVocab = DataResults['cf_to_CFVocab']

## Step 1: input: examples

In [22]:
import itertools 
import pandas as pd 
import numpy as np 
import torch 

examples = dataset[:4]

[i for i in examples]

['PID',
 'ObsDT',
 'TargetCGM.Bf24H.input_ids',
 'TargetCGM.Af2H.input_ids',
 'TimeSparse.Bf24H.timeinfo',
 'TimeSparse.Bf24H.timevalues',
 'TimeSparse.Af2H.timeinfo',
 'TimeSparse.Af2H.timevalues',
 'MedSparse.Bf24H.timestep',
 'MedSparse.Bf24H.input_ids',
 'MedSparse.Bf24H.input_wgts',
 'MedSparse.Bf24H.timeinfo',
 'MedSparse.Bf24H.timevalues',
 'MedSparse.Af2H.timestep',
 'MedSparse.Af2H.input_ids',
 'MedSparse.Af2H.input_wgts',
 'MedSparse.Af2H.timeinfo',
 'MedSparse.Af2H.timevalues',
 'EduSparse.Bf24H.timestep',
 'EduSparse.Bf24H.input_ids',
 'EduSparse.Bf24H.input_wgts',
 'EduSparse.Bf24H.timeinfo',
 'EduSparse.Bf24H.timevalues',
 'EduSparse.Af2H.timestep',
 'EduSparse.Af2H.input_ids',
 'EduSparse.Af2H.input_wgts',
 'EduSparse.Af2H.timeinfo',
 'EduSparse.Af2H.timevalues',
 'DietSparse.Bf24H.timestep',
 'DietSparse.Bf24H.input_ids',
 'DietSparse.Bf24H.input_wgts',
 'DietSparse.Bf24H.timeinfo',
 'DietSparse.Bf24H.timevalues',
 'DietSparse.Af2H.timestep',
 'DietSparse.Af2H.input_ids

## Step 2: Update Emptyness

In [23]:
import inspect 

def update_emptiness_of_examples(x, cf):
    # print(x)
    # print(cf)
    EmptyFlag = False
    values = x[cf + '.input_ids']
    # print(values[0][0])
    # print(values[0][0] == 0)
    # print(int(values[0][0]) == 0)
    # print(values)
    # print(len(values) == 1)
    # print(len(values[0]) == 1)
    # print(int(values[0][0]) == 0)
    if len(values) == 1 and len(values[0]) == 1 and int(values[0][0]) == 0:
        EmptyFlag = True
    
    # print('EmptyFlag', EmptyFlag)
    d = {}
    d[cf + '.input_ids'] = [] if EmptyFlag == True else x[cf + '.input_ids']
    if cf + '.input_wgts' in x:
        d[cf + '.input_wgts'] = [] if EmptyFlag == True else x[cf + '.input_wgts']
    d[cf + '.timestep'] = [] if EmptyFlag == True else x[cf + '.timestep']
    
    return d
                
update_emptiness_of_examples.fn_string = inspect.getsource(update_emptiness_of_examples)

In [24]:
MEDAL_Fields = EntryArgs['MEDAL_Fields']
total_MEDAL_CFs = list(itertools.chain(*[MEDAL_Fields[Field] for Field in MEDAL_Fields]))
total_MEDAL_CFs = [i for i in total_MEDAL_CFs if 'TimeSparse' not in i]
total_MEDAL_CFs

['MedSparse.Bf24H',
 'MedSparse.Af2H',
 'EduSparse.Bf24H',
 'EduSparse.Af2H',
 'DietSparse.Bf24H',
 'DietSparse.Af2H',
 'ActivitySparse.Bf24H',
 'ActivitySparse.Af2H',
 'LabVitalSparse.Bf24H',
 'LabVitalSparse.Af2H']

In [25]:
df = pd.DataFrame(examples)# [MEDAL_CFs]
for cf in total_MEDAL_CFs:
    print(cf)
    li = df.apply(lambda x: update_emptiness_of_examples(x, cf), axis = 1)
    df_new = pd.DataFrame(li.to_list())
    for col in df_new.columns:
        df[col] = df_new[col] 
df

MedSparse.Bf24H
MedSparse.Af2H
EduSparse.Bf24H
EduSparse.Af2H
DietSparse.Bf24H
DietSparse.Af2H
ActivitySparse.Bf24H
ActivitySparse.Af2H
LabVitalSparse.Bf24H
LabVitalSparse.Af2H


,PID,ObsDT,TargetCGM.Bf24H.input_ids,TargetCGM.Af2H.input_ids,TimeSparse.Bf24H.timeinfo,TimeSparse.Bf24H.timevalues,TimeSparse.Af2H.timeinfo,TimeSparse.Af2H.timevalues,MedSparse.Bf24H.timestep,MedSparse.Bf24H.input_ids,...,LabVitalSparse.Bf24H.timestep,LabVitalSparse.Bf24H.input_ids,LabVitalSparse.Bf24H.input_wgts,LabVitalSparse.Bf24H.timeinfo,LabVitalSparse.Bf24H.timevalues,LabVitalSparse.Af2H.timestep,LabVitalSparse.Af2H.input_ids,LabVitalSparse.Af2H.input_wgts,LabVitalSparse.Af2H.timeinfo,LabVitalSparse.Af2H.timevalues
0,1000672,2021-05-31,"[115, 115, 114, 113, 113, 113, 113, 112, 113, ...","[145, 140, 136, 133, 131, 128, 126, 125, 125, ...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-05-31T00:00:00, 5Min, 5, minutes, -288:T...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-05-31T00:00:00, 5Min, 5, minutes, 1:To:24]",[-143],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",...,[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-05-31T00:00:00, 5Min, 5, minutes, -288:T...",[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-05-31T00:00:00, 5Min, 5, minutes, 1:To:24]"
1,1000672,2021-06-12,"[111, 110, 109, 110, 111, 111, 111, 110, 109, ...","[124, 125, 127, 128, 129, 129, 129, 129, 129, ...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-06-12T00:00:00, 5Min, 5, minutes, -288:T...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-06-12T00:00:00, 5Min, 5, minutes, 1:To:24]",[-176],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",...,[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-06-12T00:00:00, 5Min, 5, minutes, -288:T...",[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-06-12T00:00:00, 5Min, 5, minutes, 1:To:24]"
2,1000672,2021-08-23,"[89, 89, 90, 90, 92, 94, 96, 98, 98, 98, 99, 1...","[93, 93, 94, 94, 93, 93, 93, 94, 94, 93, 93, 9...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-08-23T00:00:00, 5Min, 5, minutes, -288:T...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-08-23T00:00:00, 5Min, 5, minutes, 1:To:24]",[-196],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",...,[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-08-23T00:00:00, 5Min, 5, minutes, -288:T...",[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-08-23T00:00:00, 5Min, 5, minutes, 1:To:24]"
3,1000692,2021-09-03,"[192, 195, 198, 200, 203, 205, 206, 207, 208, ...","[196, 195, 195, 195, 193, 184, 169, 149, 129, ...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-09-03T00:00:00, 5Min, 5, minutes, -288:T...","[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-09-03T00:00:00, 5Min, 5, minutes, 1:To:24]",[-188],"[[84, 89, 95, 151, 162, 168, 174, 230]]",...,[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-09-03T00:00:00, 5Min, 5, minutes, -288:T...",[],[],[],"[ObsDTValue, TimeStepType, TimeStepSize, TimeU...","[2021-09-03T00:00:00, 5Min, 5, minutes, 1:To:24]"


In [26]:
examples = df.to_dict(orient = 'list')

## Step 2: input_ids and labels

In [27]:
examples_new = {}

In [28]:
Target_CFs = EntryArgs['Target_CFs']

df = pd.DataFrame({cf: examples[cf + '.input_ids'] for cf in Target_CFs})
df['input_ids'] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)

examples_new['input_ids'] = np.array(df['input_ids'].to_list()) # ().copy()
examples_new['labels'] = examples_new['input_ids'].copy()

## Step 3: Field to Vocab

In [29]:
import inspect 

def process_field_vocab(Field, MEDAL_Fields, cf_to_CFVocab):

    Vocab = {}

    MEDAL_CFs = MEDAL_Fields[Field]
    # adding input_ids and input_wgts
    Vocab['input_ids']  = cf_to_CFVocab[MEDAL_CFs[0]]['input_ids']
    Vocab['input_wgts'] = cf_to_CFVocab[MEDAL_CFs[0]]['input_wgts']

    # adding timestep_ids
    cf_to_CFVocab_timestep = {cf: cf_to_CFVocab[cf]['timestep_ids'] for cf in MEDAL_CFs}
    COMMON_TOKEN_list = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', ]
    idx2tkn_all = COMMON_TOKEN_list + [i.split(':')[-1] for i in list(itertools.chain(*[list(cf_to_CFVocab_timestep[cf]['tkn2tid'].keys()) for cf in MEDAL_CFs]))]
    assert len(idx2tkn_all) == len(set(idx2tkn_all))
    tid2tkn_all = {i: tkn for i, tkn in enumerate(idx2tkn_all)}
    tkn2tid_all = {tkn: i for i, tkn in enumerate(idx2tkn_all)}
    Vocab['timestep_ids'] = {'tid2tkn': tid2tkn_all, 'tkn2tid': tkn2tid_all}
    return Vocab

process_field_vocab.fn_string = inspect.getsource(process_field_vocab)

In [30]:
cf_to_CFVocab = DataResults['cf_to_CFVocab']
MEDAL_Fields = EntryArgs['MEDAL_Fields']

field_to_Vocab = {}
for Field in MEDAL_Fields:
    field_to_Vocab[Field] = process_field_vocab(Field, MEDAL_Fields, cf_to_CFVocab) 

# pprint(field_to_Vocab, sort_dicts=False, compact=True)
print(field_to_Vocab)

{'Time': {'input_ids': {'tid2tkn': {0: '[PAD]', 1: '[UNK]', 2: '[CLS]', 3: '[SEP]', 4: '[MASK]', 5: '[BOS]', 6: '[EOS]', 7: 'None', 8: 'Mon1', 9: 'Mon2', 10: 'Mon3', 11: 'Mon4', 12: 'Mon5', 13: 'Mon6', 14: 'Mon7', 15: 'Mon8', 16: 'Mon9', 17: 'Mon10', 18: 'Mon11', 19: 'Mon12', 20: 'WOY1', 21: 'WOY2', 22: 'WOY3', 23: 'WOY4', 24: 'WOY5', 25: 'WOY6', 26: 'WOY7', 27: 'WOY8', 28: 'WOY9', 29: 'WOY10', 30: 'WOY11', 31: 'WOY12', 32: 'WOY13', 33: 'WOY14', 34: 'WOY15', 35: 'WOY16', 36: 'WOY17', 37: 'WOY18', 38: 'WOY19', 39: 'WOY20', 40: 'WOY21', 41: 'WOY22', 42: 'WOY23', 43: 'WOY24', 44: 'WOY25', 45: 'WOY26', 46: 'WOY27', 47: 'WOY28', 48: 'WOY29', 49: 'WOY30', 50: 'WOY31', 51: 'WOY32', 52: 'WOY33', 53: 'WOY34', 54: 'WOY35', 55: 'WOY36', 56: 'WOY37', 57: 'WOY38', 58: 'WOY39', 59: 'WOY40', 60: 'WOY41', 61: 'WOY42', 62: 'WOY43', 63: 'WOY44', 64: 'WOY45', 65: 'WOY46', 66: 'WOY47', 67: 'WOY48', 68: 'WOY49', 69: 'WOY50', 70: 'WOY51', 71: 'WOY52', 72: 'WOY53', 73: 'Weekday0', 74: 'Weekday1', 75: 'Weekda

## Step 4: MEDAL Field's CFs

### 4.1 Get Field's CFs

In [31]:
##############################
Field = 'Med'
# Field = 'Diet'
# Field = 'Time'
##############################

MEDAL_Fields = EntryArgs['MEDAL_Fields']
MEDAL_CFs = MEDAL_Fields[Field]
MEDAL_CFs

['MedSparse.Bf24H', 'MedSparse.Af2H']

In [32]:
MEDAL_CFs = MEDAL_Fields[Field]
seqtype_list = ['input_ids', 'input_wgts', 'timestep_ids']

### 4.2 input_ids

In [33]:
MEDAL_CFs

['MedSparse.Bf24H', 'MedSparse.Af2H']

In [34]:
seqtype = 'input_ids'

df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
# df

In [35]:
df[seqtype] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
lst = df[seqtype].to_list()
df

,MedSparse.Bf24H,MedSparse.Af2H,input_ids
0,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",[],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ..."
1,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",[],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ..."
2,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...",[],"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ..."
3,"[[84, 89, 95, 151, 162, 168, 174, 230]]",[],"[[84, 89, 95, 151, 162, 168, 174, 230]]"


In [36]:
field_to_Vocab[Field][seqtype]

{'tid2tkn': {0: '[PAD]',
  1: '[UNK]',
  2: '[CLS]',
  3: '[SEP]',
  4: '[MASK]',
  5: '[BOS]',
  6: '[EOS]',
  7: 'f1:unk',
  8: 'f1:FrequencyValue_unk',
  9: 'f1:FrequencyValue_minor',
  10: 'f1:FrequencyValue_1',
  11: 'f1:FrequencyValue_1,1',
  12: 'f1:FrequencyValue_1,2',
  13: 'f1:FrequencyValue_1,32',
  14: 'f1:FrequencyValue_1,64',
  15: 'f1:FrequencyValue_1,8',
  16: 'f1:FrequencyValue_1,4',
  17: 'f1:FrequencyValue_1,16',
  18: 'f1:FrequencyValue_1,33',
  19: 'f1:FrequencyValue_1,9',
  20: 'f1:FrequencyValue_1,42',
  21: 'f1:FrequencyValue_1,1024',
  22: 'f1:FrequencyValue_1,5',
  23: 'f1:FrequencyValue_1,16777216',
  24: 'f1:FrequencyValue_1,1048576',
  25: 'f1:FrequencyValue_1,134217728',
  26: 'f1:FrequencyValue_1,37',
  27: 'f1:FrequencyValue_1,536870912',
  28: 'f1:FrequencyValue_1,36',
  29: 'f1:FrequencyValue_1,128',
  30: 'f1:FrequencyValue_1,32768',
  31: 'f1:FrequencyValue_1,131072',
  32: 'f1:FrequencyValue_1,17',
  33: 'f1:FrequencyValue_1,3',
  34: 'f1:FrequencyV

### 4.3 input_wgts

In [37]:
seqtype = 'input_wgts'

if examples[MEDAL_CFs[0] + f'.{seqtype}'][0] is None:
    lst = None 
    print(lst, 'no wgt is available, or all wgt are 1')
else:
    df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
    df[seqtype] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
    lst = df[seqtype].to_list()
df

,MedSparse.Bf24H,MedSparse.Af2H,input_wgts
0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[],"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
1,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[],"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
2,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[],"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
3,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]",[],"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]"


### 4.4 timesteps

In [38]:
seqtype = 'timestep'
df = pd.DataFrame({cf: [i for i in examples[cf + f'.{seqtype}']]
                   for cf in MEDAL_CFs})
df

,MedSparse.Bf24H,MedSparse.Af2H
0,[-143],[]
1,[-176],[]
2,[-196],[]
3,[-188],[]


### 4.5 fn to get raw ids

In [39]:
import inspect 

def process_field_input_ids(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab):

    field_info = {}
    MEDAL_CFs = MEDAL_Fields[Field]

    seqtype = 'input_ids'
    df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
    df[seqtype] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
    lst = df[seqtype].to_list()
    # lst = pad_with_numpy(lst)
    field_info[seqtype] = lst
    seqtype = 'input_wgts'

    if examples[MEDAL_CFs[0] + f'.{seqtype}'][0] is None:
        lst = None 
        # print(lst, 'no wgt is available, or all wgt are 1')
    else:
        df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
        df[seqtype] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
        lst = df[seqtype].to_list()
        # lst = pad_with_numpy(lst)
        # print(lst.shape)
    field_info[seqtype] = lst

    seqtype = 'timestep'
    df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
    df[seqtype] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
        
    lst = df[seqtype].to_list()
    field_info[seqtype] = lst
    
    return field_info


process_field_input_ids.fn_string = inspect.getsource(process_field_input_ids)


In [40]:
field_info = process_field_input_ids(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
pprint(field_info, sort_dicts=False, compact=True)

{'input_ids': [[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89,
                 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89,
                 95, 147, 158, 164, 193, 226, 95, 147, 158, 164, 173, 226, 95,
                 147, 158, 164, 190, 226, 95, 147, 158, 164, 202, 226, 95, 147,
                 158, 164, 173, 226, 95, 147, 158, 164, 179, 226, 96, 147, 158,
                 164, 188, 226, 95, 147, 158, 164, 173, 226, 95, 150, 161, 164,
                 179, 229, 95, 148, 159, 164, 179, 227, 96, 148, 159, 164, 188,
                 227, 95, 148, 159, 164, 186, 227, 95, 149, 160, 164, 219, 226,
                 95, 149, 160, 164, 173, 226, 95, 149, 160, 164, 179, 228, 95,
                 149, 160, 164, 186, 226]],
               [[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89,
                 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 95,
                 147, 158, 164, 193, 226, 95, 147, 158, 164, 190, 226, 95, 147

### 4.6 fn to get timestep info

In [41]:

import inspect 

def get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab):

    FieldVocab = field_to_Vocab[Field]
    field_list = MEDAL_Fields[Field]
    total_index_list = []
    total_timedelta_info = []

    TimeUnit = None
    TimeStep = None
    for field in field_list:
        timeinfo_col = f'{field}.timeinfo'
        timevalues_col = f'{field}.timevalues'
        d = dict(zip(examples[timeinfo_col][0], examples[timevalues_col][0]))
        # if TimeUnit is not None:
        #     TimeUnit = d['TimeUnit']
        # else:
        #     assert TimeUnit == d['TimeUnit']
        # print(d)
        TimeUnit = d['TimeUnit'] 
        TimeStep = d['TimeStepSize']
        

        TimeStepType = d['TimeStepType']
        StartIdx_To_EndIdx_columns = [i for i in d if 'StartIdx-To-EndIdx' in i]
        
        for StartIdx_To_EndIdx_col in StartIdx_To_EndIdx_columns:
            start_to_end = d[StartIdx_To_EndIdx_col].split(':To:')
            StartIdx = int(start_to_end[0])
            EndIdx   = int(start_to_end[1])
            index_list = [f'{TimeStepType}_{i}' for i in list(range(StartIdx, EndIdx + 1))]
            timedelta_info = [(TimeStep, TimeUnit)] * len(index_list)
            total_index_list = total_index_list + index_list
            total_timedelta_info = total_timedelta_info + timedelta_info

            
    timesteps = total_index_list
    timestep_ids = [FieldVocab['timestep_ids']['tkn2tid'][i] for i in total_index_list]
    timesteps_orig_ids = [int(i.split('_')[-1]) for i in timesteps]

    timestep_info = {
        'timesteps': timesteps,
        'timestep_ids': timestep_ids, 
        'timesteps_orig_ids': timesteps_orig_ids, 
        'total_timedelta_info': total_timedelta_info,
    }
    return timestep_info

get_timesteps_info.fn_string = inspect.getsource(get_timesteps_info)

In [42]:
timestep_info = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
timestep_ids = timestep_info['timestep_ids']

### 4.7 Complete Field Info

In [43]:
UNK_ID = 1
PAD_ID = 0

default_template_dict = {
    'input_ids':  [[UNK_ID]] * len(timestep_ids),
    'input_wgts': [[1]     ] * len(timestep_ids), 
    'timestep_ids': timestep_ids.copy()
}

In [44]:
df = pd.DataFrame(field_info)
df

,input_ids,input_wgts,timestep
0,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-143]
1,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-176]
2,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-196]
3,"[[84, 89, 95, 151, 162, 168, 174, 230]]","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]",[-188]


In [45]:
[i for i in default_template_dict]

['input_ids', 'input_wgts', 'timestep_ids']

In [46]:
[i for i in timestep_info]

['timesteps', 'timestep_ids', 'timesteps_orig_ids', 'total_timedelta_info']

In [47]:
timesteps_orig_ids = timestep_info['timesteps_orig_ids']
# timesteps_orig_ids.index()

In [48]:
def update_seqtype_base_on_timestep(single_data_point, default_template_dict, timestep_info):

    default_template_dict = default_template_dict.copy()

    timesteps_orig_ids = timestep_info['timesteps_orig_ids']
    
    updated_data_point = {}

    timestep = single_data_point['timestep']

    seqtype = 'input_ids'
    seqtype_values = single_data_point[seqtype]
    seqtype_values_template = default_template_dict[seqtype]
    for idx, timestep_id in enumerate(timestep):
        tensor_loc_idx = timesteps_orig_ids.index(timestep_id)
        # print(idx, timestep_id, tensor_loc_idx)
        seqtype_values_template[tensor_loc_idx] = seqtype_values[idx]
        # print(seqtype_values[idx])
        # print(tensor_loc_idx)
        # print(seqtype_values_template[tensor_loc_idx])

    updated_data_point[seqtype] = seqtype_values_template

    seqtype = 'input_wgts'
    seqtype_values = single_data_point[seqtype]
    seqtype_values_template = default_template_dict[seqtype]
    for idx, timestep_id in enumerate(timestep):
        seqtype_values_template[timestep_id] = seqtype_values[idx]
    # updated_data_point[seqtype] = pad_with_numpy(seqtype_values_template)
    updated_data_point[seqtype] = seqtype_values_template

    # updated_data_point['timestep_ids'] = pad_with_numpy(default_template_dict['timestep_ids'])
    updated_data_point['timestep_ids'] = default_template_dict['timestep_ids']
    return updated_data_point

update_seqtype_base_on_timestep.fn_string = inspect.getsource(update_seqtype_base_on_timestep)

In [49]:
import inspect 
import numpy as np 

def pad_with_numpy(lst, fill_value=0):
    # Base case: Assume all elements at the deepest level are numbers
    while isinstance(lst[0], list):
        max_len = max(len(x) for x in lst)
        lst = [x + [fill_value] * (max_len - len(x)) for x in lst]
        lst = [pad_with_numpy(x, fill_value) for x in lst]
    # Convert list to array
    return np.array(lst)

pad_with_numpy.fn_string = inspect.getsource(pad_with_numpy)

In [50]:
# single_data_point = df.iloc[0]
# updated_data_point = update_seqtype_base_on_timestep(single_data_point, default_template_dict)
# # updated_data_point
# # for k, v in updated_data_point.items(): print(k, v.shape)

In [51]:
li = df.apply(lambda x: update_seqtype_base_on_timestep(x, default_template_dict, timestep_info), axis = 1)
field_info_final = pd.DataFrame(li.to_list()).to_dict(orient = 'list')
for k, v in field_info_final.items():
    print(k, pad_with_numpy(v).shape)
    field_info_final[k] = pad_with_numpy(v)


input_ids (4, 313, 136)
input_wgts (4, 313, 136)
timestep_ids (4, 313)


In [52]:
for k, v in field_info_final.items():
    print(k, v.shape)

input_ids (4, 313, 136)
input_wgts (4, 313, 136)
timestep_ids (4, 313)


In [53]:
values = field_info_final['timestep_ids']
values

array([[  4,   5,   6, ..., 314, 315, 316],
       [  4,   5,   6, ..., 314, 315, 316],
       [  4,   5,   6, ..., 314, 315, 316],
       [  4,   5,   6, ..., 314, 315, 316]])

In [54]:
df = pd.DataFrame(field_info)
df

,input_ids,input_wgts,timestep
0,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-143]
1,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-176]
2,"[[84, 89, 84, 89, 84, 89, 84, 89, 84, 89, 84, ...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",[-196]
3,"[[84, 89, 95, 151, 162, 168, 174, 230]]","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]",[-188]


In [55]:
288 - 204

84

In [56]:
field_info_final['input_ids'][0][84]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [57]:
for k, v in field_info_final.items():
    print(k, v.shape)

input_ids (4, 313, 136)
input_wgts (4, 313, 136)
timestep_ids (4, 313)


## Step 5: Time Field's CFs 

In [58]:
##############################
# Field = 'Diet'
Field = 'Time'
##############################

MEDAL_Fields = EntryArgs['MEDAL_Fields']
MEDAL_CFs = MEDAL_Fields[Field]
MEDAL_CFs

['TimeSparse.Bf24H', 'TimeSparse.Af2H']

In [59]:
[i for i in examples]

['PID',
 'ObsDT',
 'TargetCGM.Bf24H.input_ids',
 'TargetCGM.Af2H.input_ids',
 'TimeSparse.Bf24H.timeinfo',
 'TimeSparse.Bf24H.timevalues',
 'TimeSparse.Af2H.timeinfo',
 'TimeSparse.Af2H.timevalues',
 'MedSparse.Bf24H.timestep',
 'MedSparse.Bf24H.input_ids',
 'MedSparse.Bf24H.input_wgts',
 'MedSparse.Bf24H.timeinfo',
 'MedSparse.Bf24H.timevalues',
 'MedSparse.Af2H.timestep',
 'MedSparse.Af2H.input_ids',
 'MedSparse.Af2H.input_wgts',
 'MedSparse.Af2H.timeinfo',
 'MedSparse.Af2H.timevalues',
 'EduSparse.Bf24H.timestep',
 'EduSparse.Bf24H.input_ids',
 'EduSparse.Bf24H.input_wgts',
 'EduSparse.Bf24H.timeinfo',
 'EduSparse.Bf24H.timevalues',
 'EduSparse.Af2H.timestep',
 'EduSparse.Af2H.input_ids',
 'EduSparse.Af2H.input_wgts',
 'EduSparse.Af2H.timeinfo',
 'EduSparse.Af2H.timevalues',
 'DietSparse.Bf24H.timestep',
 'DietSparse.Bf24H.input_ids',
 'DietSparse.Bf24H.input_wgts',
 'DietSparse.Bf24H.timeinfo',
 'DietSparse.Bf24H.timevalues',
 'DietSparse.Af2H.timestep',
 'DietSparse.Af2H.input_ids

In [60]:
# seqtype = 'timeinfo'
# df = pd.DataFrame({cf: examples[cf + f'.{seqtype}'] for cf in MEDAL_CFs})
# df['ObsDT']

In [61]:
timestep_info = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
[i for i in timestep_info]

['timesteps', 'timestep_ids', 'timesteps_orig_ids', 'total_timedelta_info']

In [62]:
field_to_Vocab[Field]

{'input_ids': {'tid2tkn': {0: '[PAD]',
   1: '[UNK]',
   2: '[CLS]',
   3: '[SEP]',
   4: '[MASK]',
   5: '[BOS]',
   6: '[EOS]',
   7: 'None',
   8: 'Mon1',
   9: 'Mon2',
   10: 'Mon3',
   11: 'Mon4',
   12: 'Mon5',
   13: 'Mon6',
   14: 'Mon7',
   15: 'Mon8',
   16: 'Mon9',
   17: 'Mon10',
   18: 'Mon11',
   19: 'Mon12',
   20: 'WOY1',
   21: 'WOY2',
   22: 'WOY3',
   23: 'WOY4',
   24: 'WOY5',
   25: 'WOY6',
   26: 'WOY7',
   27: 'WOY8',
   28: 'WOY9',
   29: 'WOY10',
   30: 'WOY11',
   31: 'WOY12',
   32: 'WOY13',
   33: 'WOY14',
   34: 'WOY15',
   35: 'WOY16',
   36: 'WOY17',
   37: 'WOY18',
   38: 'WOY19',
   39: 'WOY20',
   40: 'WOY21',
   41: 'WOY22',
   42: 'WOY23',
   43: 'WOY24',
   44: 'WOY25',
   45: 'WOY26',
   46: 'WOY27',
   47: 'WOY28',
   48: 'WOY29',
   49: 'WOY30',
   50: 'WOY31',
   51: 'WOY32',
   52: 'WOY33',
   53: 'WOY34',
   54: 'WOY35',
   55: 'WOY36',
   56: 'WOY37',
   57: 'WOY38',
   58: 'WOY39',
   59: 'WOY40',
   60: 'WOY41',
   61: 'WOY42',
   62: 'WOY4

In [63]:
# examples

import inspect 

def extract_datetime_components_as_list(dt):
    return [
        # f'Y{dt.year}',
        f'Mon{dt.month}',
        f'WOY{dt.isocalendar().week}',
        f'Weekday{dt.weekday()}',
        f'Hour{dt.hour}',
        f'Min{dt.minute}',
        f'Sec{dt.second}'
    ]
extract_datetime_components_as_list.fn_string = inspect.getsource(extract_datetime_components_as_list)



timesteps_orig_ids = timestep_info['timesteps_orig_ids']
timedelta_info = timestep_info['total_timedelta_info']
df = pd.DataFrame(
    {
        'ObsDT': examples['ObsDT'], 
        'timesteps_orig_ids': [timesteps_orig_ids] * len(examples['ObsDT']), 
        'timedelta_info': [timedelta_info] * len(examples['ObsDT']),
        'timestep_ids': [timestep_info['timestep_ids']] * len(examples['ObsDT'])
    }
)
df['datetime'] = df.apply(lambda x: [x['ObsDT'] + pd.Timedelta(i*int(delta[0]), unit=delta[-1]) 
                                     for i, delta in zip(x['timesteps_orig_ids'], x['timedelta_info'])], axis=1)
df['inputs'] = df['datetime'].apply(lambda x: [extract_datetime_components_as_list(i) for i in x])
df['input_ids'] = df['inputs'].apply(lambda x: [[field_to_Vocab[Field]['input_ids']['tkn2tid'][j] for j in i] for i in x])
df

,ObsDT,timesteps_orig_ids,timedelta_info,timestep_ids,datetime,inputs,input_ids
0,2021-05-31,"[-288, -287, -286, -285, -284, -283, -282, -28...","[(5, minutes), (5, minutes), (5, minutes), (5,...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[2021-05-30 00:00:00, 2021-05-30 00:05:00, 202...","[[Mon5, WOY21, Weekday6, Hour0, Min0, Sec0], [...","[[12, 40, 79, 80, 104, 164], [12, 40, 79, 80, ..."
1,2021-06-12,"[-288, -287, -286, -285, -284, -283, -282, -28...","[(5, minutes), (5, minutes), (5, minutes), (5,...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[2021-06-11 00:00:00, 2021-06-11 00:05:00, 202...","[[Mon6, WOY23, Weekday4, Hour0, Min0, Sec0], [...","[[13, 42, 77, 80, 104, 164], [13, 42, 77, 80, ..."
2,2021-08-23,"[-288, -287, -286, -285, -284, -283, -282, -28...","[(5, minutes), (5, minutes), (5, minutes), (5,...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[2021-08-22 00:00:00, 2021-08-22 00:05:00, 202...","[[Mon8, WOY33, Weekday6, Hour0, Min0, Sec0], [...","[[15, 52, 79, 80, 104, 164], [15, 52, 79, 80, ..."
3,2021-09-03,"[-288, -287, -286, -285, -284, -283, -282, -28...","[(5, minutes), (5, minutes), (5, minutes), (5,...","[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...","[2021-09-02 00:00:00, 2021-09-02 00:05:00, 202...","[[Mon9, WOY35, Weekday3, Hour0, Min0, Sec0], [...","[[16, 54, 76, 80, 104, 164], [16, 54, 76, 80, ..."


In [64]:
field_info_final = df[['input_ids', 'timestep_ids']].to_dict(orient='list')
for k, v in field_info_final.items():
    # print(k, pad_with_numpy(v).shape)
    field_info_final[k] = pad_with_numpy(v)


for k, v in field_info_final.items():
    print(k, v.shape)

input_ids (4, 313, 6)
timestep_ids (4, 313)


## Step 6: Transform Function

In [65]:
def extract_datetime_components_as_list(dt):
    return [
        # f'Y{dt.year}',
        f'Mon{dt.month}',
        f'WOY{dt.isocalendar().week}',
        f'Weekday{dt.weekday()}',
        f'Hour{dt.hour}',
        f'Min{dt.minute}',
        f'Sec{dt.second}'
    ]
         
extract_datetime_components_as_list.fn_string = inspect.getsource(extract_datetime_components_as_list)

In [66]:
  


def transform_fn(examples):
    examples_tfm = {}

    ################# Target's input_ids and labels
    Target_CFs = EntryArgs['Target_CFs']
    df = pd.DataFrame({cf: examples[cf + '.input_ids'] for cf in Target_CFs})
    df['input_ids'] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
    # examples_tfm['input_ids'] = torch.LongTensor(    np.array(df['input_ids'].to_list()    )) # ().copy()
    # examples_tfm['labels']    = torch.LongTensor(    np.array(df['input_ids'].to_list()    ))
    examples_tfm['input_ids'] = np.array(df['input_ids'].to_list()    ) # ().copy()
    examples_tfm['labels']    = np.array(df['input_ids'].to_list()    )


    
    total_fields_tfm = {}
    ################# Target's input_ids and labels
    for Field in MEDAL_Fields: 

        if Field != 'Time':
            field_info = process_field_input_ids(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
            timestep_info   = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
            timestep_ids = timestep_info['timestep_ids']
            UNK_ID = 1; PAD_ID = 0
            default_template_dict = {
                'input_ids':  [[UNK_ID]] * len(timestep_ids),
                'input_wgts': [[1]     ] * len(timestep_ids), 
                'timestep_ids': timestep_ids.copy()
            }
            df = pd.DataFrame(field_info)
            li = df.apply(lambda x: update_seqtype_base_on_timestep(x, default_template_dict, timestep_info), axis = 1)
            field_info_final = pd.DataFrame(li.to_list()).to_dict(orient = 'list')
            for k, v in field_info_final.items():
                # print(k, pad_with_numpy(v).shape)
                if '_wgts' not in k:
                    # field_info_final[k] = torch.LongTensor(pad_with_numpy(v))
                    field_info_final[k] = pad_with_numpy(v)
                else:
                    # field_info_final[k] = torch.FloatTensor(pad_with_numpy(v))
                    field_info_final[k] = pad_with_numpy(v)
            total_fields_tfm[f'Field_{Field}'] = field_info_final

            
        elif Field == 'Time':

            timestep_info = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
            timesteps_orig_ids = timestep_info['timesteps_orig_ids']
            timedelta_info = timestep_info['total_timedelta_info']

            d = {
                'ObsDT': examples['ObsDT'], 
                'timesteps_orig_ids': [timesteps_orig_ids] * len(examples['ObsDT']), 
                'timedelta_info': [timedelta_info] * len(examples['ObsDT']),
                'timestep_ids': [timestep_info['timestep_ids']] * len(examples['ObsDT'])
            }
            df = pd.DataFrame(d)
            df['datetime'] = df.apply(lambda x: [x['ObsDT'] + pd.Timedelta(i*int(delta[0]), unit=delta[-1]) 
                                                for i, delta in zip(x['timesteps_orig_ids'], x['timedelta_info'])], axis=1)
            df['inputs'] = df['datetime'].apply(lambda x: [extract_datetime_components_as_list(i) for i in x])
            df['input_ids'] = df['inputs'].apply(lambda x: [[field_to_Vocab[Field]['input_ids']['tkn2tid'][j] for j in i] for i in x])

            field_info_final = df[['input_ids', 'timestep_ids']].to_dict(orient='list')
            for k, v in field_info_final.items():
                # print(k, pad_with_numpy(v).shape)
                # field_info_final[k] = pad_with_numpy(v)
                # field_info_final[k] = torch.LongTensor(pad_with_numpy(v))
                field_info_final[k] = pad_with_numpy(v)
                

            total_fields_tfm[f'Field_{Field}'] = field_info_final

        else:
            raise ValueError(f"Invalid Field: {Field}")
        
        
    examples_tfm['total_field_info'] = total_fields_tfm
    return examples_tfm

In [67]:
examples_tfm = transform_fn(examples)
examples_tfm

{'input_ids': array([[115, 115, 114, ..., 129, 129, 128],
        [111, 110, 109, ..., 120, 119, 118],
        [ 89,  89,  90, ...,  92,  92,  93],
        [192, 195, 198, ..., 145, 148, 150]]),
 'labels': array([[115, 115, 114, ..., 129, 129, 128],
        [111, 110, 109, ..., 120, 119, 118],
        [ 89,  89,  90, ...,  92,  92,  93],
        [192, 195, 198, ..., 145, 148, 150]]),
 'total_field_info': {'Field_Time': {'input_ids': array([[[ 12,  40,  79,  80, 104, 164],
           [ 12,  40,  79,  80, 109, 164],
           [ 12,  40,  79,  80, 114, 164],
           ...,
           [ 12,  41,  73,  81, 154, 164],
           [ 12,  41,  73,  81, 159, 164],
           [ 12,  41,  73,  82, 104, 164]],
   
          [[ 13,  42,  77,  80, 104, 164],
           [ 13,  42,  77,  80, 109, 164],
           [ 13,  42,  77,  80, 114, 164],
           ...,
           [ 13,  42,  78,  81, 154, 164],
           [ 13,  42,  78,  81, 159, 164],
           [ 13,  42,  78,  82, 104, 164]],
   
        

## Step 7: Final Entry Function

In [68]:
import inspect 
import numpy as np
import pandas as pd 
import itertools
import torch
from recfldtkn.loadtools import convert_variables_to_pystirng


def fn_entry_method_for_finaldata(dataset, cf_to_CFVocab, EntryArgs):
    
    ###########################
    Target_CFs = EntryArgs['Target_CFs']
    MEDAL_Fields = EntryArgs['MEDAL_Fields']
        
    field_to_Vocab = {}
    for Field in MEDAL_Fields:
        field_to_Vocab[Field] = process_field_vocab(Field, MEDAL_Fields, cf_to_CFVocab) 


    def transform_fn(examples):
        examples_tfm = {}
        ################# Target's input_ids and labels
        Target_CFs = EntryArgs['Target_CFs']
        df = pd.DataFrame({cf: examples[cf + '.input_ids'] for cf in Target_CFs})
        df['input_ids'] = df.apply(lambda x: list(itertools.chain(*x.values)), axis=1)
        examples_tfm['input_ids'] = torch.LongTensor(    np.array(df['input_ids'].to_list()    )) # ().copy()
        examples_tfm['labels']    = torch.LongTensor(    np.array(df['input_ids'].to_list()    ))
        # examples_tfm['input_ids'] = np.array(df['input_ids'].to_list()    ) # ().copy()
        # examples_tfm['labels']    = np.array(df['input_ids'].to_list()    )


        total_fields_tfm = {}
        ################# Target's input_ids and labels
        for Field in MEDAL_Fields: 

            if Field != 'Time':
                field_info = process_field_input_ids(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
                timestep_info   = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
                timestep_ids = timestep_info['timestep_ids']
                UNK_ID = 1; PAD_ID = 0
                default_template_dict = {
                    'input_ids':  [[UNK_ID]] * len(timestep_ids),
                    'input_wgts': [[1]     ] * len(timestep_ids), 
                    'timestep_ids': timestep_ids.copy()
                }
                df = pd.DataFrame(field_info)
                li = df.apply(lambda x: update_seqtype_base_on_timestep(x, default_template_dict, timestep_info), axis = 1)
                field_info_final = pd.DataFrame(li.to_list()).to_dict(orient = 'list')
                for k, v in field_info_final.items():
                    # print(k, pad_with_numpy(v).shape)
                    if '_wgts' not in k:
                        field_info_final[k] = torch.LongTensor(pad_with_numpy(v))
                        # field_info_final[k] = pad_with_numpy(v)
                    else:
                        field_info_final[k] = torch.Tensor(pad_with_numpy(v))
                        # field_info_final[k] = pad_with_numpy(v)
                total_fields_tfm[f'Field_{Field}'] = field_info_final

                
            elif Field == 'Time':

                timestep_info = get_timesteps_info(examples, Field, MEDAL_Fields, field_to_Vocab, cf_to_CFVocab)
                timesteps_orig_ids = timestep_info['timesteps_orig_ids']
                timedelta_info = timestep_info['total_timedelta_info']

                d = {
                    'ObsDT': examples['ObsDT'], 
                    'timesteps_orig_ids': [timesteps_orig_ids] * len(examples['ObsDT']), 
                    'timedelta_info': [timedelta_info] * len(examples['ObsDT']),
                    'timestep_ids': [timestep_info['timestep_ids']] * len(examples['ObsDT'])
                }
                df = pd.DataFrame(d)
                df['datetime'] = df.apply(lambda x: [x['ObsDT'] + pd.Timedelta(i*int(delta[0]), unit=delta[-1]) 
                                                    for i, delta in zip(x['timesteps_orig_ids'], x['timedelta_info'])], axis=1)
                df['inputs'] = df['datetime'].apply(lambda x: [extract_datetime_components_as_list(i) for i in x])
                df['input_ids'] = df['inputs'].apply(lambda x: [[field_to_Vocab[Field]['input_ids']['tkn2tid'][j] for j in i] for i in x])

                field_info_final = df[['input_ids', 'timestep_ids']].to_dict(orient='list')
                for k, v in field_info_final.items():
                    # print(k, pad_with_numpy(v).shape)
                    # field_info_final[k] = pad_with_numpy(v)
                    field_info_final[k] = torch.LongTensor(pad_with_numpy(v))
                    # field_info_final[k] = pad_with_numpy(v)
                    

                total_fields_tfm[f'Field_{Field}'] = field_info_final

            else:
                raise ValueError(f"Invalid Field: {Field}")
            
        examples_tfm['total_field_info'] = total_fields_tfm
        return examples_tfm
    
    dataset.set_transform(transform_fn)
    results = {'dataset': dataset, 'field_to_Vocab': field_to_Vocab}
    return results


fn_entry_method_for_finaldata.fn_string = inspect.getsource(fn_entry_method_for_finaldata)


prefix = ['import pandas as pd', 
          'import numpy as np',
          'import itertools',
          'import torch',
          ]
fn_variables = [
    pad_with_numpy,
    process_field_vocab, 
    process_field_input_ids,
    get_timesteps_info,
    update_seqtype_base_on_timestep,
    extract_datetime_components_as_list,
    fn_entry_method_for_finaldata,
    ]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{ENTRY_METHOD}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline\fn_learning\EntryForCgmMedalNtp.py


In [69]:
EntryArgs['MEDAL_Fields']

{'Time': ['TimeSparse.Bf24H', 'TimeSparse.Af2H'],
 'Med': ['MedSparse.Bf24H', 'MedSparse.Af2H'],
 'Edu': ['EduSparse.Bf24H', 'EduSparse.Af2H'],
 'Diet': ['DietSparse.Bf24H', 'DietSparse.Af2H'],
 'Activity': ['ActivitySparse.Bf24H', 'ActivitySparse.Af2H'],
 'Lab': ['LabVitalSparse.Bf24H', 'LabVitalSparse.Af2H']}

In [70]:
results = fn_entry_method_for_finaldata(dataset, cf_to_CFVocab, EntryArgs)
dataset = results['dataset']
dataset

Dataset({
    features: ['PID', 'ObsDT', 'TargetCGM.Bf24H.input_ids', 'TargetCGM.Af2H.input_ids', 'TimeSparse.Bf24H.timeinfo', 'TimeSparse.Bf24H.timevalues', 'TimeSparse.Af2H.timeinfo', 'TimeSparse.Af2H.timevalues', 'MedSparse.Bf24H.timestep', 'MedSparse.Bf24H.input_ids', 'MedSparse.Bf24H.input_wgts', 'MedSparse.Bf24H.timeinfo', 'MedSparse.Bf24H.timevalues', 'MedSparse.Af2H.timestep', 'MedSparse.Af2H.input_ids', 'MedSparse.Af2H.input_wgts', 'MedSparse.Af2H.timeinfo', 'MedSparse.Af2H.timevalues', 'EduSparse.Bf24H.timestep', 'EduSparse.Bf24H.input_ids', 'EduSparse.Bf24H.input_wgts', 'EduSparse.Bf24H.timeinfo', 'EduSparse.Bf24H.timevalues', 'EduSparse.Af2H.timestep', 'EduSparse.Af2H.input_ids', 'EduSparse.Af2H.input_wgts', 'EduSparse.Af2H.timeinfo', 'EduSparse.Af2H.timevalues', 'DietSparse.Bf24H.timestep', 'DietSparse.Bf24H.input_ids', 'DietSparse.Bf24H.input_wgts', 'DietSparse.Bf24H.timeinfo', 'DietSparse.Bf24H.timevalues', 'DietSparse.Af2H.timestep', 'DietSparse.Af2H.input_ids', 'DietSp

In [71]:
examples = dataset[:8]
examples

{'input_ids': tensor([[115, 115, 114,  ..., 129, 129, 128],
         [111, 110, 109,  ..., 120, 119, 118],
         [ 89,  89,  90,  ...,  92,  92,  93],
         ...,
         [192, 198, 200,  ..., 139, 137, 136],
         [192, 186, 180,  ..., 102,  99,  96],
         [203, 203, 204,  ..., 211, 212, 214]]),
 'labels': tensor([[115, 115, 114,  ..., 129, 129, 128],
         [111, 110, 109,  ..., 120, 119, 118],
         [ 89,  89,  90,  ...,  92,  92,  93],
         ...,
         [192, 198, 200,  ..., 139, 137, 136],
         [192, 186, 180,  ..., 102,  99,  96],
         [203, 203, 204,  ..., 211, 212, 214]]),
 'total_field_info': {'Field_Time': {'input_ids': tensor([[[ 12,  40,  79,  80, 104, 164],
            [ 12,  40,  79,  80, 109, 164],
            [ 12,  40,  79,  80, 114, 164],
            ...,
            [ 12,  41,  73,  81, 154, 164],
            [ 12,  41,  73,  81, 159, 164],
            [ 12,  41,  73,  82, 104, 164]],
   
           [[ 13,  42,  77,  80, 104, 164],
    